![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/visual-nlp/2.1.Pdf_processing.ipynb)

## Blogposts and videos

- [Text Detection in Spark OCR](https://medium.com/spark-nlp/text-detection-in-spark-ocr-dcd8002bdc97)

- [Table Detection & Extraction in Spark OCR](https://medium.com/spark-nlp/table-detection-extraction-in-spark-ocr-50765c6cedc9)

- [Extract Tabular Data from PDF in Spark OCR](https://medium.com/spark-nlp/extract-tabular-data-from-pdf-in-spark-ocr-b02136bc0fcb)

- [Signature Detection in Spark OCR](https://medium.com/spark-nlp/signature-detection-in-spark-ocr-32f9e6f91e3c)

- [GPU image pre-processing in Spark OCR](https://medium.com/spark-nlp/gpu-image-pre-processing-in-spark-ocr-3-1-0-6fc27560a9bb)

- [How to Setup Spark OCR on UBUNTU - Video](https://www.youtube.com/watch?v=cmt4WIcL0nI)


**More examples here**

https://github.com/JohnSnowLabs/spark-ocr-workshop

For get the trial license please go to:

https://www.johnsnowlabs.com/install/

### Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
!pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install(refresh_install=True, visual=True)

In [ ]:
from johnsnowlabs import nlp, visual
import pandas as pd

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start(visual=True)

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8356 (8).json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.2.2, 💊Spark-Healthcare==5.2.1, 🕶Spark-OCR==5.1.2, running on ⚡ PySpark==3.1.2


In [ ]:
import pkg_resources

from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

## Read pdfs to the dataframe and display

In [ ]:
pdf_path = visual.pkg_resources.resource_filename('sparkocr', 'resources/ocr/pdfs/*.pdf')
pdf_example_df = spark.read.format("binaryFile").load(pdf_path).cache()

visual.display_pdf(pdf_example_df)

## Define pipeline for extract text from the searchable and ocr text from the scaned pdf

In [ ]:
def pipeline():

    # If text PDF extract text
    pdf_to_text = visual.PdfToText() \
        .setInputCol("content") \
        .setOutputCol("text") \
        .setSplitPage(True) \
        .setExtractCoordinates(True) \
        .setStoreSplittedPdf(True)

    # If image pdf, extract image
    pdf_to_image = visual.PdfToImage() \
        .setInputCol("content") \
        .setOutputCol("image") \
        .setKeepInput(True)

    # Run OCR
    ocr = visual.ImageToText() \
        .setInputCol("image") \
        .setOutputCol("text") \
        .setConfidenceThreshold(60)

    pipeline = PipelineModel(stages=[
        pdf_to_text,
        pdf_to_image,
        ocr
    ])

    return pipeline

## Run pipeline and show results

In [ ]:
result = pipeline().transform(pdf_example_df).cache()
result.show()

+--------------------+--------------------+------+--------------------+--------------------+----------------+---------------+--------------------+--------------------+-----------+-------+-----------+--------------------+---------+
|                path|    modificationTime|length|                text|           positions|height_dimension|width_dimension|             content|               image|total_pages|pagenum|documentnum|          confidence|exception|
+--------------------+--------------------+------+--------------------+--------------------+----------------+---------------+--------------------+--------------------+-----------+-------+-----------+--------------------+---------+
|file:/usr/local/l...|2024-02-06 19:52:...| 70556|Alexandria was fo...|[{[{A, 72.024, 76...|             792|            612|[25 50 44 46 2D 3...|                null|          0|      0|          0|-1.79769313486231...|     null|
|file:/usr/local/l...|2024-02-06 19:52:...|693743|Patient Name
Fina...|[{[{P

## Display text using pandas dataframe

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

result.select("text").toPandas().style.set_properties(**{'white-space': 'pre-wrap', 'text-align': 'left'})